<a href="https://colab.research.google.com/github/masterlyj/self-LLM_Agent_RL/blob/main/run_training_dpo_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Pipeline
[run_training_dpo_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

Cloning into 'MedicalGPT'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 99 (delta 19), reused 52 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (99/99), 8.98 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/MedicalGPT
build_domain_tokenizer.py   reward_modeling.py
chatpdf.py                  role_play_data/
CITATION.cff                run_dpo.sh
_config.yml                 run_eval_quantize.sh
CONTRIBUTING.md             run_full_sft.sh
convert_dataset.py          run_grpo.sh
data/                       run_grpo_xm_test.sh
DISCLAIMER                  run_orpo.sh
docs/                       run_ppo.sh
dpo_training.py             run_pt.sh
eval_quantize.py            run_quant.sh
fastapi_server_demo.py      run_rm.sh
gradio_demo.py              run_sft_accelerate.sh
grpo_training.py            run_sft.sh
inference_multigpu_demo.py  run_training_dpo_pi

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [2]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [6]:
!python pretraining.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --bf16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 50 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2026-01-23 05:27:53.640607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769146073.660074    3657 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769146073.666170    3657 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769146073.681287    3657 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146073.681313    3657 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146073.681317    3657 computation_placer.cc:177] computation placer alr

In [7]:
%ls -lh outputs-pt-v1

total 22M
-rw-r--r-- 1 root root 1.1K Jan 23 05:38 adapter_config.json
-rw-r--r-- 1 root root  17M Jan 23 05:38 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jan 23 05:38 added_tokens.json
-rw-r--r-- 1 root root  472 Jan 23 05:38 all_results.json
-rw-r--r-- 1 root root 2.4K Jan 23 05:38 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Jan 23 05:37 checkpoint-750/
drwxr-xr-x 2 root root 4.0K Jan 23 05:37 checkpoint-800/
drwxr-xr-x 2 root root 4.0K Jan 23 05:38 checkpoint-844/
-rw-r--r-- 1 root root  263 Jan 23 05:38 eval_results.json
-rw-r--r-- 1 root root 1.6M Jan 23 05:38 merges.txt
-rw-r--r-- 1 root root 5.1K Jan 23 05:38 README.md
drwxr-xr-x 3 root root 4.0K Jan 23 05:28 runs/
-rw-r--r-- 1 root root  616 Jan 23 05:38 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Jan 23 05:38 tokenizer_config.json
-rw-r--r-- 1 root root  20K Jan 23 05:38 trainer_state.json
-rw-r--r-- 1 root root  229 Jan 23 05:38 train_results.json
-rw-r--r-- 1 root root 3.3M Jan 23 05:38 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [8]:
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B --lora_model outputs-pt-v1 --output_dir merged-pt/

2026-01-23 05:40:41.079561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769146841.100656    6979 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769146841.106963    6979 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769146841.122868    6979 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146841.122893    6979 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146841.122897    6979 computation_placer.cc:177] computation placer alr

In [9]:
%ls -lh merged-pt/

total 958M
-rw-r--r-- 1 root root  605 Jan 23 05:40 added_tokens.json
-rw-r--r-- 1 root root 2.4K Jan 23 05:40 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Jan 23 05:40 config.json
-rw-r--r-- 1 root root  117 Jan 23 05:40 generation_config.json
-rw-r--r-- 1 root root 1.6M Jan 23 05:40 merges.txt
-rw-r--r-- 1 root root 943M Jan 23 05:40 model.safetensors
-rw-r--r-- 1 root root  616 Jan 23 05:40 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Jan 23 05:40 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jan 23 05:40 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jan 23 05:40 vocab.json


In [10]:
%cat merged-pt/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [11]:
%ls ./data/finetune

medical_sft_1K_format.jsonl        sharegpt_zh_1K_format.jsonl
numina_cot_sharegpt_data_1k.jsonl


In [12]:
import json

with open('./data/finetune/numina_cot_sharegpt_data_1k.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 3:  # 只显示前3行
            break
        print(f"\n第 {i+1} 行：")
        try:
            data = json.loads(line.strip())
            print(json.dumps(data, indent=2, ensure_ascii=False))
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
            print(f"原始行内容: {line.strip()}")

with open('./data/finetune/medical_sft_1K_format.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 3:  # 只显示前3行
            break
        print(f"\n第 {i+1} 行：")
        try:
            data = json.loads(line.strip())
            print(json.dumps(data, indent=2, ensure_ascii=False))
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
            print(f"原始行内容: {line.strip()}")


第 1 行：
{
  "id": "AI-MO/NuminaMath-CoT-0",
  "conversations": [
    {
      "from": "human",
      "value": "Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$. Let's think step by step and output the final answer within \\boxed{}."
    },
    {
      "from": "gpt",
      "value": "For an arithmetic sequence, the difference between consecutive terms must be equal. Therefore, we can set up the following equations based on the sequence given:\n\\[ (y + 2) - \\left(-\\frac{1}{3}\\right) = 4y - (y+2) \\]\n\nSimplify and solve these equations:\n\\[ y + 2 + \\frac{1}{3} = 4y - y - 2 \\]\n\\[ y + \\frac{7}{3} = 3y - 2 \\]\n\\[ \\frac{7}{3} + 2 = 3y - y \\]\n\\[ \\frac{13}{3} = 2y \\]\n\\[ y = \\frac{13}{6} \\]\n\nThus, the value of $y$ that satisfies the given arithmetic sequence is $\\boxed{\\frac{13}{6}}$."
    }
  ]
}

第 2 行：
{
  "id": "AI-MO/NuminaMath-CoT-1",
  "conversations": [
    {
      "from": "human",
      "value": "Suppose that $g(x) 

In [13]:
import json

# 1. 定义文件路径
problem_file = './data/finetune/numina_cot_sharegpt_data_1k.jsonl'

# 2. 读取、处理并写回（覆盖原文件，建议先备份）
fixed_lines = []
with open(problem_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line.strip())
        # 移除'id'字段
        if 'id' in data:
            del data['id']
        # 确保格式正确，只保留 conversations
        fixed_lines.append(json.dumps({'conversations': data['conversations']}, ensure_ascii=False))

with open(problem_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(fixed_lines))

print(f"文件 ‘{problem_file}’ 已修复，多余的 ‘id’ 字段已移除。")

文件 ‘./data/finetune/numina_cot_sharegpt_data_1k.jsonl’ 已修复，多余的 ‘id’ 字段已移除。


In [14]:
!python supervised_finetuning.py \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2026-01-23 05:41:05.860097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769146865.879676    7096 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769146865.885547    7096 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769146865.899961    7096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146865.899985    7096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769146865.899989    7096 computation_placer.cc:177] computation placer alr

In [15]:
%ls -lh outputs-sft-v1

total 22M
-rw-r--r-- 1 root root 1.1K Jan 23 05:53 adapter_config.json
-rw-r--r-- 1 root root  17M Jan 23 05:53 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jan 23 05:53 added_tokens.json
-rw-r--r-- 1 root root  431 Jan 23 05:53 all_results.json
-rw-r--r-- 1 root root 2.4K Jan 23 05:53 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Jan 23 05:53 checkpoint-250/
-rw-r--r-- 1 root root  221 Jan 23 05:53 eval_results.json
-rw-r--r-- 1 root root 1.6M Jan 23 05:53 merges.txt
-rw-r--r-- 1 root root 5.1K Jan 23 05:53 README.md
drwxr-xr-x 3 root root 4.0K Jan 23 05:41 runs/
-rw-r--r-- 1 root root  648 Jan 23 05:53 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Jan 23 05:53 tokenizer_config.json
-rw-r--r-- 1 root root 6.0K Jan 23 05:53 trainer_state.json
-rw-r--r-- 1 root root  230 Jan 23 05:53 train_results.json
-rw-r--r-- 1 root root 3.3M Jan 23 05:53 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [16]:
!python merge_peft_adapter.py \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

2026-01-23 05:54:03.752934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769147643.772447   10414 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769147643.778378   10414 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769147643.793344   10414 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769147643.793375   10414 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769147643.793379   10414 computation_placer.cc:177] computation placer alr

In [17]:
%ls -lh merged-sft/

total 958M
-rw-r--r-- 1 root root  605 Jan 23 05:54 added_tokens.json
-rw-r--r-- 1 root root 2.4K Jan 23 05:54 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Jan 23 05:54 config.json
-rw-r--r-- 1 root root  117 Jan 23 05:54 generation_config.json
-rw-r--r-- 1 root root 1.6M Jan 23 05:54 merges.txt
-rw-r--r-- 1 root root 943M Jan 23 05:54 model.safetensors
-rw-r--r-- 1 root root  616 Jan 23 05:54 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Jan 23 05:54 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jan 23 05:54 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jan 23 05:54 vocab.json


In [18]:
%cat merged-sft/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage2 SFT训练完成。

# Stage 3: DPO(Direct Preference Optimization)

第三阶段：DPO(Direct Preference Optimization)直接偏好优化，DPO通过直接优化语言模型来实现对其行为的精确控制，而无需使用复杂的强化学习，也可以有效学习到人类偏好，DPO相较于RLHF更容易实现且易于训练，效果更好

| Stage 3: Direct Preference Optimization        |  [dpo_training.py](https://github.com/shibing624/MedicalGPT/blob/main/dpo_training.py) | [run_dpo.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_dpo.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是`Qwen/Qwen2.5-0.5B` 或者 Stage2得到的SFT模型
2. 数据集：DPO阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [19]:
%ls ./data/reward/

dpo_zh_500.jsonl


In [20]:
!python dpo_training.py \
    --model_name_or_path ./merged-sft \
    --template_name qwen \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 500 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 256 \
    --max_target_length 256 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --bf16 True \
    --fp16 False \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache  \
    --optim adamw_torch

2026-01-23 05:54:28.667425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769147668.686665   10531 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769147668.692407   10531 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769147668.706891   10531 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769147668.706915   10531 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769147668.706919   10531 computation_placer.cc:177] computation placer alr

In [21]:
%ls -lh outputs-dpo-v1

total 22M
-rw-r--r-- 1 root root 1.1K Jan 23 06:23 adapter_config.json
-rw-r--r-- 1 root root  17M Jan 23 06:23 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jan 23 06:23 added_tokens.json
-rw-r--r-- 1 root root  765 Jan 23 06:24 all_results.json
-rw-r--r-- 1 root root 2.4K Jan 23 06:23 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Jan 23 06:23 checkpoint-100/
drwxr-xr-x 2 root root 4.0K Jan 23 06:09 checkpoint-50/
-rw-r--r-- 1 root root  570 Jan 23 06:24 eval_results.json
-rw-r--r-- 1 root root 1.6M Jan 23 06:23 merges.txt
-rw-r--r-- 1 root root 2.4K Jan 23 06:23 README.md
drwxr-xr-x 3 root root 4.0K Jan 23 05:54 runs/
-rw-r--r-- 1 root root  648 Jan 23 06:23 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Jan 23 06:23 tokenizer_config.json
-rw-r--r-- 1 root root  57K Jan 23 06:23 trainer_state.json
-rw-r--r-- 1 root root 6.7K Jan 23 06:23 training_args.bin
-rw-r--r-- 1 root root  229 Jan 23 06:23 train_results.json
-rw-r--r-- 1 root root 3.3M Jan 23 06:23 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [22]:
!python merge_peft_adapter.py \
    --base_model merged-sft --lora_model outputs-dpo-v1 --output_dir merged-dpo/

2026-01-23 06:24:54.393382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769149494.412753   18345 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769149494.418865   18345 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769149494.433791   18345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149494.433820   18345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149494.433824   18345 computation_placer.cc:177] computation placer alr

In [23]:
%ls -lh merged-dpo/

total 958M
-rw-r--r-- 1 root root  605 Jan 23 06:25 added_tokens.json
-rw-r--r-- 1 root root 2.4K Jan 23 06:25 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Jan 23 06:25 config.json
-rw-r--r-- 1 root root  117 Jan 23 06:25 generation_config.json
-rw-r--r-- 1 root root 1.6M Jan 23 06:25 merges.txt
-rw-r--r-- 1 root root 943M Jan 23 06:25 model.safetensors
-rw-r--r-- 1 root root  616 Jan 23 06:25 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Jan 23 06:25 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jan 23 06:25 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jan 23 06:25 vocab.json


In [24]:
%cat merged-dpo/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage3 偏好建模第一次训练完成。

**至此一个完整的训练流程演示完成。**

# Test

In [25]:
!python inference.py --base_model merged-dpo
# 或在shell中运行
# python inference.py --base_model merged-dpo --interactive

2026-01-23 06:25:38.746777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769149538.766046   18547 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769149538.772040   18547 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769149538.786482   18547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149538.786504   18547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149538.786508   18547 computation_placer.cc:177] computation placer alr

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。


In [26]:
!python inference.py --base_model Qwen/Qwen2.5-0.5B

2026-01-23 06:27:55.167146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769149675.193839   19139 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769149675.199975   19139 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769149675.215339   19139 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149675.215375   19139 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769149675.215379   19139 computation_placer.cc:177] computation placer alr